In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca
import folium
import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

import datetime as dt
import time

from calitp import get_engine

engine = get_engine()
connection = engine.connect()

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Download/Process Tract-level Job Data
Source: [Urban Institute](https://datacatalog.urban.org/dataset/longitudinal-employer-household-dynamics-origin-destination-employment-statistics-lodes) 

Code adapted from [Bus Service Increase](https://github.com/cal-itp/data-analyses/blob/main/bus_service_increase/create_calenviroscreen_lehd_data.py#L107-L110)

In [22]:
# Download LEHD data from Urban Institute

URBAN_URL = "https://urban-data-catalog.s3.amazonaws.com/drupal-root-live/"
DATE_UPDATE = "2022/03/11/"

lehd_prv = (pd.read_csv(f"{URBAN_URL}{DATE_UPDATE}wac_all_tract_minus_fed.csv")
>> filter(_.st==6, _.year==_.year.max())
>> select(_["st":"c000"])
>> rename(jobs_prv = "c000")
           )

lehd_fed = (pd.read_csv(f"{URBAN_URL}{DATE_UPDATE}wac_fed_tract.csv")
>> filter(_.st==6, _.year==_.year.max())
>> select(_["st":"c000"])
>> rename(jobs_fed = "c000")
           )

In [30]:
lehd_all = (lehd_prv 
            >> full_join(_,lehd_fed)
            >> mutate(jobs_fed=_.jobs_fed.fillna(0),
                      jobs_total=_.jobs_prv+_.jobs_fed
                     )
           )

lehd_all = (lehd_all
            .assign(geo_id = lehd_all.apply(lambda x: '0' + str(x.trct), axis = 1).astype(str))
           )
lehd_all.sample(10)

,st,stname,cty,ctyname,trct,trctname,year,jobs_prv,jobs_fed,jobs_total,geo_id
6701,6,California,6081,"San Mateo County, CA",6081604400,"6044 (San Mateo, CA)",2019,3601,3.0,3604.0,06081604400
5370,6,California,6071,"San Bernardino County, CA",6071002028,"20.28 (San Bernardino, CA)",2019,2129,0.0,2129.0,06071002028
5191,6,California,6067,"Sacramento County, CA",6067009010,"90.10 (Sacramento, CA)",2019,24277,10.0,24287.0,06067009010
6549,6,California,6077,"San Joaquin County, CA",6077004308,"43.08 (San Joaquin, CA)",2019,261,0.0,261.0,06077004308
5633,6,California,6071,"San Bernardino County, CA",6071012400,"124 (San Bernardino, CA)",2019,19365,54.0,19419.0,06071012400
2467,6,California,6037,"Los Angeles County, CA",6037432102,"4321.02 (Los Angeles, CA)",2019,1034,0.0,1034.0,06037432102
5219,6,California,6067,"Sacramento County, CA",6067009325,"93.25 (Sacramento, CA)",2019,1054,0.0,1054.0,06067009325
7968,6,California,6111,"Ventura County, CA",6111008700,"87 (Ventura, CA)",2019,3778,67.0,3845.0,06111008700
3432,6,California,6037,"Los Angeles County, CA",6037920029,"9200.29 (Los Angeles, CA)",2019,405,0.0,405.0,06037920029
4652,6,California,6065,"Riverside County, CA",6065042517,"425.17 (Riverside, CA)",2019,104,0.0,104.0,06065042517


In [31]:
# save

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

lehd_all.to_parquet(f"{GCS_FILE_PATH}job_density")